In [1]:
import torch
import math
import matplotlib.pyplot as plt
from torch import nn
import numpy as np
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from numba import njit, cuda
from torch import nn
from torch.nn import functional as F

In [8]:
a = torch.tensor([[1,2,3],[4,5,6]])
b = torch.tensor([1,2,3])
a*b

tensor([[ 1,  4,  9],
        [ 4, 10, 18]])

In [37]:
class BC_2D:
    def __init__(self, left, right, up, down):
        """
        Args:
            left, right: (alpha, beta, f(t))
        """
        # alpha*u + beta*u_x + gamma*u_y = f(t)
        self.left_alpha, self.left_beta, self.left_func = left
        self.right_alpha, self.right_beta, self.right_func = right
        self.up_alpha, self.up_beta, self.up_func = up
        self.down_alpha, self.down_beta, self.down_func = down


    def apply(self, simu):
        gamma_left = self.left_beta / simu.dx
        gamma_right = self.right_beta / simu.dx
        gamma_up = self.up_beta / simu.dx
        gamma_down = self.down_beta / simu.dx
        
        # Get coordinates
        x_coord = torch.arange(simu.x_grid, requires_grad=False, device=simu.device).expand(simu.y_grid, simu.x_grid) * simu.dx
        y_coord = torch.arange(simu.y_grid, requires_grad=False, device=simu.device).unsqueeze(1).expand(simu.y_grid, simu.x_grid) * simu.dx

        # Left boundary
        # print(simu.grid[1:-1,0].shape)
        # print(y_coord[0,:].shape)

        simu.grid[1:-1,0] = (self.left_func(x_coord[0,:], y_coord[:,0], simu.cur_time) - gamma_left * simu.grid[1:-1,1]) / (self.left_alpha - gamma_left)


        # Right boundary
        simu.grid[1:-1,-1] = (self.right_func(x_coord[0,:], y_coord[:,0], simu.cur_time) + gamma_right * simu.grid[1:-1,-2]) / (self.right_alpha + gamma_right)

        # Left boundary
        simu.grid[0,1:-1] = (self.up_func(x_coord[0,:], y_coord[:,0], simu.cur_time) - gamma_up * simu.grid[1,1:-1]) / (self.up_alpha - gamma_up)

        # Down boundary
        simu.grid[-1,1:-1] = (self.down_func(x_coord[0,:], y_coord[:,0], simu.cur_time) + gamma_down * simu.grid[-2,1:-1]) / (self.down_alpha + gamma_down)
        
class ContConduct:
    def __init__(self, c_func):
        self.c_func = c_func
        self.map = None

    def make_conduct_map(self, simu):
        # Initialize conduct map
        self.map = torch.zeros(simu.grid.shape[0], simu.grid.shape[1], device=simu.device, dtype=simu.dtype)
        # Get coordinates
        x_coord = torch.arange(simu.x_grid, requires_grad=False, device=simu.device).expand(simu.y_grid, simu.x_grid) * simu.dx
        y_coord = torch.arange(simu.y_grid, requires_grad=False, device=simu.device).unsqueeze(1).expand(simu.y_grid, simu.x_grid) * simu.dx

        # Apply conductivity for interior
        self.map[1:-1,1:-1] = self.c_func(x_coord, y_coord)

        # Apply conductivity for boundary
        self.map[0,:] = self.map[1,:] # up
        self.map[-1,:] = self.map[-2,:] # down
        self.map[:,0] = self.map[:,1] # left
        self.map[:,-1] = self.map[:,-2] # right

        # Compute mean conductivity
        # left
        self.map_left = 2 * self.map[1:-1,1:-1] * self.map[0:-2, 1:-1] / (self.map[1:-1,1:-1] + self.map[0:-2, 1:-1])

        # right
        self.map_right = 2 * self.map[1:-1,1:-1] * self.map[2:, 1:-1] / (self.map[1:-1,1:-1] + self.map[2, 1:-1])

        # up
        self.map_up = 2 * self.map[1:-1,1:-1] * self.map[1:-1, 0:-2] / (self.map[1:-1,1:-1] + self.map[1:-1, 0:-2])

        # down
        self.map_down = 2 * self.map[1:-1,1:-1] * self.map[1:-1, 2:] / (self.map[1:-1,1:-1] + self.map[1:-1, 2:])

        self.merge_map = torch.stack([
            self.map_left,
            self.map_right,
            self.map_up,
            self.map_down
        ], dim=0).unsqueeze(0)
        
class Heat2dSimu:
    def __init__(self, map_shape, dx, total_time, tstep, bc, ic, c, plot_step, Q=0, device='cpu', do_progress_bar=True, dtype=torch.float32):
        """
        Args:
            map_shape (tuple): Physical size of the 2D domain.
            step (float): Step size of *interior* points (excluding boundaries).
            total_time (float): End time for the simulation.
            tstep (int): Step size of time.
            bc (iterable): Boundary condition with 4 elements. Order: up, r down, left, right.
            ic (callable): Function for initial condition.
            c (float): Diffusion coefficient.
            plot_step (int): How often (in steps) to plot the solution.
            device (str): 'cpu' or 'cuda', which device to use for Tensor operations.
        """
        self.grid = None
        self.grid_bach = None
        self.map_shape = map_shape
        self.dx = dx
        self.total_time = total_time
        self.dt = tstep
        self.bc = bc
        self.ic = ic
        self.c = c
        self.plot_step = plot_step
        self.do_progress_bar = do_progress_bar
        self.Q = self.make_heat_source_func(Q)
        self.device = device
        self.dtype = dtype
        self.conv = None
        self.decide_computation_mode()
        self.cur_time = 0




        # Check device
        if torch.cuda.is_available() and device != 'cpu':
            self.device = device
            print('Your simulation will be performed based on CUDA.')
        else:
            self.device = 'cpu'
            print('Your simulation will be performed based on CPU.')

        self.make_grid()

        # Useful preload data
        self.x_coord_tensor = torch.arange(self.x_grid, requires_grad=False, device=self.device).expand(self.y_grid, self.x_grid) * self.dx
        self.y_coord_tensor = torch.arange(self.y_grid, requires_grad=False, device=self.device).unsqueeze(1).expand(self.y_grid, self.x_grid) * self.dx

        # Some initialization
        self.set_ic()

    def make_heat_source_func(self, Q):
        if callable(Q):
            return Q
        else:
            def func(x, y, t):
                return Q
            return func

    def set_ic(self):
        # print(self.grid[1:-1,1:-1].shape)
        self.grid[1:-1,1:-1] = self.ic(self.x_coord_tensor, self.y_coord_tensor)

    def set_bc(self):
        self.bc.apply(self)


    def make_grid(self):
        # Get size of grid
        self.x_grid = math.ceil(self.map_shape[1] / self.dx)
        self.y_grid = math.ceil(self.map_shape[0] / self.dx)
        self.grid = torch.zeros(self.y_grid+2, self.x_grid+2, dtype=self.dtype, device=self.device)

        # For convenience, prevent overhead for unsqueeze
        self.grid_ch = self.grid.unsqueeze(0).unsqueeze(0).expand(1,1,-1,-1)

    def make_conv_core_continuous(self):
        self.conv = nn.Conv2d(1, 4, kernel_size=(3,3), bias=False, device=self.device, dtype=self.dtype)
        dt_dx2 = self.dt / (self.dx**2)
        kernel = torch.tensor([
            [[ [0,0,0], [1,-1,0], [0,0,0] ]],
            [[ [0,0,0], [0,-1,1], [0,0,0] ]],
            [[ [0,1,0], [0,-1,0], [0,0,0] ]],
            [[ [0,0,0], [0,-1,0], [0,1,0] ]]
        ], device=self.device, dtype=self.dtype) * dt_dx2

        with torch.no_grad():
            self.conv.weight[:] = kernel

    def make_conv_core_const(self):
        self.conv = nn.Conv2d(1, 1, kernel_size=(3,3), bias=False, device=self.device, dtype=self.dtype)
        dt_dx2 = self.dt / (self.dx**2)
        kernel = torch.tensor([
            [[ [0,1,0], [1,-4,1], [0,1,0] ]],
        ], device=self.device, dtype=self.dtype) * dt_dx2 * self.c

        with torch.no_grad():
            self.conv.weight[:] = kernel

    def decide_computation_mode(self):
        if isinstance(self.c, ContConduct):
            self.update = self.update_continuous
            self.make_conv_core_continuous()
        else:
            self.update = self.update_const
            self.make_conv_core_const()

    def update_continuous(self):
        with torch.inference_mode():
            diff_map = self.conv(self.grid_ch)
            diff = (torch.sum(diff_map * self.c.merge_map, dim=1, keepdim=True)
                    + self.Q(self.x_coord_tensor, self.y_coord_tensor, self.cur_time)) * self.dt
            self.grid_ch[:, :, 1:-1, 1:-1] += diff


    def update_const(self):

        diff = self.conv(self.grid_ch)
        # print(diff.shape)
        self.grid_ch[:, :, 1:-1, 1:-1] += diff + self.Q(self.x_coord_tensor, self.y_coord_tensor, self.cur_time) * self.dt

    def start(self):
        saved = []
        append = saved.append
        cur_max = -float('inf')
        cur_min = float('inf')
        with torch.inference_mode():
            for step in tqdm(range( int(self.total_time/self.dt) ),disable=False):
                self.set_bc()
                self.update()
                self.cur_time += self.dt
                
                if step % self.plot_step == 0:
                    copied = self.grid[1:-1,1:-1].clone().to('cpu')
                    if self.dtype == torch.bfloat16:
                        copied = copied.to(dtype=torch.float32)
                    append(copied)
                    
                    this_max = torch.max(copied)
                    if cur_max < this_max:
                        cur_max = this_max
                        
                    this_min = torch.min(copied)
                    if cur_min > this_min:
                        cur_min = this_min
        
        # Append the very final result          
        copied = self.grid[1:-1,1:-1].clone().to('cpu')
        if self.dtype == torch.bfloat16:
            copied = copied.to(dtype=torch.float32)
        append(copied)

        fig, axis = plt.subplots()
        pcm = axis.pcolormesh(self.grid.cpu().numpy()[1:-1,1:-1], cmap=plt.cm.jet,
                              vmin=float(cur_min), vmax=float(cur_max))
        plt.colorbar(pcm, ax=axis)
        axis.set_xlabel('x grids')
        axis.set_ylabel('y grids')
        
        for i, data in enumerate(saved):
            pcm.set_array(data.numpy())
            axis.set_title(f'Distribution at t={i * self.plot_step * self.dt:.4f}')
            plt.pause(0.01)

        plt.show()

SyntaxError: invalid syntax (1996178436.py, line 245)

In [35]:
if torch.max( torch.tensor([1,2,3])) <4:
    print('yes')

yes


In [39]:
# %%timeit -r 10 -n 10
map_shape=(1,torch.pi)
dx = 0.01
total_time=1
dt=0.00001

def ic(x,y):
    return torch.sin(x)
c=1
plot_step=1000
def Q(x,y,t):
    return 0
device='cpu'
factor = dt*c*2/dx**2
print(factor)
def func(x,y):
    return x+y
con = ContConduct(func)
def func2(x,y,t):
    return 0
def func3(x,y,t):
    return 0
bc= BC_2D((1,0,func2),(1,0,func2),(0,1,func3),(0,1,func3))
test = Heat2dSimu(map_shape, dx, total_time, dt, bc, ic, c, plot_step, Q=0, device='cuda', do_progress_bar=True, dtype=torch.float64)
con.make_conduct_map(test)
# con.map_right.shape
test.start()


0.2
Your simulation will be performed based on CUDA.


  0%|          | 0/99999 [00:00<?, ?it/s]


TypeError: func2() missing 2 required positional arguments: 'y' and 't'

In [27]:
result = test.grid[1,1:-1]
max(result)

tensor(0.3711, device='cuda:0', dtype=torch.float64)

In [38]:
import math
x = torch.linspace(0, torch.pi-dx, result.shape[0], device='cuda')
exact = (torch.sin(x) * math.exp(-total_time)).to(device='cuda')


In [42]:
float(torch.norm(exact-result) / int(result.shape[0]))

tensor(0.0002, device='cuda:0', dtype=torch.float64)

In [19]:
height = 4  # number of rows (y-axis)
width = 5   # number of columns (x-axis)

x_coord_tensor = torch.arange(width).expand(height, width)
y_coord_tensor = torch.arange(height).unsqueeze(1).expand(height, width)
print(x_coord_tensor)
print(y_coord_tensor)

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])
tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]])


In [27]:
import torch

height = 4  # number of rows (y-axis)
width = 5   # number of columns (x-axis)

y_coord_tensor = torch.arange(height).unsqueeze(1).repeat(1, width)
print(y_coord_tensor)

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]])


In [4]:
torch.manual_seed(42)
big = torch.tensor([
    [[ [0,0,0], [1,-1,0], [0,0,0] ]],
    [[ [0,0,0], [0,-1,1], [0,0,0] ]],
    [[ [0,1,0], [0,-1,0], [0,0,0] ]],
    [[ [0,0,0], [0,-1,0], [0,1,0] ]]
], device='cuda', dtype=torch.float32)


In [6]:
small_merge = nn.Conv2d(1, 4, kernel_size=(3,3), bias=False, device='cuda')
small_one = nn.Conv2d(1, 1, kernel_size=(3,3), bias=False, device='cuda')
tensor = torch.rand((1,1,1000,1000), device='cuda', requires_grad=False)
small_left = nn.Conv2d(1, 1, kernel_size=(1,3), bias=False, device='cuda')
small_right = nn.Conv2d(1, 1, kernel_size=(1,3), bias=False, device='cuda')
small_up = nn.Conv2d(1, 1, kernel_size=(3,1), bias=False, device='cuda')
small_down = nn.Conv2d(1, 1, kernel_size=(3,1), bias=False, device='cuda')
with torch.no_grad():
    small_left.weight[:] = torch.tensor([
        [[[1, -1, 0]]],
    ], dtype=torch.float32, device='cuda')
    small_right.weight[:] = torch.tensor([
        [[[0, -1, 1]]],
    ], dtype=torch.float32, device='cuda')
    small_up.weight[:] = torch.tensor([
        [[[1], [-1], [0]]]
    ], device='cuda', dtype=torch.float32)
    small_up.weight[:] = torch.tensor([
        [[[0], [-1], [1]]]
    ], device='cuda', dtype=torch.float32)
    small_merge.weight[:] = big
    small_one.weight[:] = torch.tensor([
        [[ [1,0,0], [1,-4,1], [0,1,0] ]]
    ], device='cuda', dtype=torch.float32)

In [189]:
%%timeit
c = small_merge(tensor)
torch.sum(c, dim=1).squeeze(0)


158 μs ± 8.55 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit -r 100 -n 10000
tensor.unsqueeze(0).unsqueeze(0)

2.19 μs ± 141 ns per loop (mean ± std. dev. of 100 runs, 10,000 loops each)


In [40]:
%%timeit
left = small_left(tensor[:,:,1:-1,:])
right = small_right(tensor[:,:,1:-1,:])
up = small_up(tensor[:,:,:,1:-1])
down = small_down(tensor[:,:,:,1:-1])
(left + right + up + down).squeeze(0).squeeze(0)

NameError: name 'small_left' is not defined

In [159]:
c = small_merge(tensor)
torch.sum(c, dim=1).shape

torch.Size([1, 998, 998])

In [115]:
a = small1(tensor) + small2(tensor)
res = small3(tensor)
print(a == res[:,0,:,:] + res[:,1,:,:])

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]]], device='cuda:0')


In [126]:
r = torch.tensor([[1,2,3],[4,5,6]])
r_batch = r.unsqueeze(0).unsqueeze(0).expand(1,2,-1,-1)
r[0,0] = 10
r_batch[0,0,0,1] = 14
print(r)
print(r_batch)

tensor([[10, 14,  3],
        [ 4,  5,  6]])
tensor([[[[10, 14,  3],
          [ 4,  5,  6]],

         [[10, 14,  3],
          [ 4,  5,  6]]]])


1.01 μs ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

# Manually set weights (must match shape)
with torch.no_grad():
    conv.weight.copy_(torch.tensor([[[[0, 0, 0],
                                      [1, -1, 0],
                                      [0, 0, 0]]]]))

# Input tensor
x = torch.randn(1, 1, 5, 5)
output = conv(x)

print(output.shape)

torch.Size([1, 1, 3, 3])
